Importing the Dependencies

In [6]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [7]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [8]:
# setup kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 966MB/s]


In [10]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [11]:
# unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [12]:
!ls # to see the extracted file

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the Dataset

In [13]:
data = pd.read_csv('IMDB Dataset.csv')

In [14]:
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data.tail() # for printing the last five rows

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [17]:
# How is the data is distributed?
data['sentiment'].value_counts()

# there is no class imbalance

,count
sentiment,
positive,25000
negative,25000


In [18]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

<ipython-input-18-2164277587>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

In [21]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Pre-Processing

In [22]:
# tokenize text data
tokenizer = Tokenizer(num_words=5000) # coverts words/phrases into numbers (we're going to take the most common 5000 words in this data)
tokenizer.fit_on_texts(train_data['review'])
word_index = tokenizer.word_index
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [23]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [24]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [26]:
import numpy as np

embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

embedding_dim = 100
vocab_size = len(word_index) + 1

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [29]:
import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=200,
                    trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model(np.zeros((1, 200)).astype(np.int32))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (1, 200, 100)          │    11,228,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (1, 100)               │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 1)                 │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,308,601 (43.14 MB)

 Trainable params: 80,501 (314.46 KB)

 Non-trainable params: 11,228,100 (42.83 MB)

In [30]:
# trainig the model
model.fit(x_train, y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 417ms/step - accuracy: 0.6304 - loss: 0.6345 - val_accuracy: 0.8077 - val_loss: 0.4352
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 261s 416ms/step - accuracy: 0.7809 - loss: 0.4649 - val_accuracy: 0.8453 - val_loss: 0.3556
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 417ms/step - accuracy: 0.8262 - loss: 0.3894 - val_accuracy: 0.8583 - val_loss: 0.3246
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 417ms/step - accuracy: 0.8427 - loss: 0.3588 - val_accuracy: 0.8662 - val_loss: 0.3079
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 264s 420ms/step - accuracy: 0.8504 - loss: 0.3400 - val_accuracy: 0.8741 - val_loss: 0.2933


Model Evaluation

In [31]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 75ms/step - accuracy: 0.8704 - loss: 0.2948
Test loss: 0.2979612648487091
Test accuracy: 0.869700014591217


Building Predictive System

In [32]:
def predict_sentiment(review):
  # tokenize and pad the review text
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen = 200)

  prediction = model.predict(padded_sequence)

  sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'

  return sentiment

In [33]:
# example usage

new_review = "The movie was not only good, but the acting was also amazing!"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step
The sentiment of the review is: positive
